In [ ]:
# !pip install splinter
# !pip install selenium
# !pip install webdriver-manager

In [4]:
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
import requests
import pprint
from itertools import chain

In [ ]:
def get_brewery_api():
    i = 0
    url = f'https://api.openbrewerydb.org/breweries?page={i}&per_page=200'
    brewery_list = []
    for i in range(1,44):
        response = requests.get(url)
        brewery_data = response.json()
        brewery_list.append(brewery_data)
        time.sleep(1)
    all_brewerys_pd = list(chain.from_iterable(brewery_list))
    return all_brewerys_pd

In [ ]:
get_brewery_api()

In [5]:
def scrape_untapped(brewery_type):
    '''Scrape the untapped website to collect the latest beer information
    open the website'''
    executable_path = {'executable_path': ChromeDriverManager().install()}
    browser = Browser('chrome', **executable_path, headless=False)
    #reading the html code
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')

    all_brewery_info = []

    url_untapped = f'https://untappd.com/brewery/top_rated?brewery_type={brewery_type}'
    browser.visit(url_untapped)
    time.sleep(5)
    breweries = browser.find_by_xpath('//*[@id="slide"]/div[2]/div/div[1]/div/div[4]')
    for brewery in breweries:
        title_link = brewery.text
        all_brewery_info.append(title_link)
        time.sleep(2)


    brewery_list = all_brewery_info[0].split("\n")
    
    brewery_list_all = []

    
#     brewery_list.remove("Report Ad")
    i = 0
    while i < len(brewery_list):
        if i == "Report Ad":
            brewery_list.remove("Report Ad")
        else:
         try:
            brewery = brewery_list[i:i+5]
            #print(brewery)
            dictionary = dict()
            dictionary['brewery_name'] = brewery[0]
            dictionary['brewery_location'] = brewery[1]
            dictionary['brewery_type'] = brewery[2]
            dictionary['number_beers'] = brewery[3]
            dictionary['brewery_ratings'] = brewery[4]
         except IndexError:
            print(f"This batch broke: {brewery}")
            break
        brewery_list_all.append(dictionary)
        i += 5
    browser.quit()
    return brewery_list_all


In [6]:
scrape_untapped('macro_brewery')

[{'brewery_name': 'Founders Brewing Co.',
  'brewery_location': 'Grand Rapids, MI United States',
  'brewery_type': 'Macro Brewery',
  'number_beers': '966 Beers 7,170,018 Ratings',
  'brewery_ratings': '(3.962)'},
 {'brewery_name': 'Wicked Weed Brewing',
  'brewery_location': 'Asheville, NC United States',
  'brewery_type': 'Macro Brewery',
  'number_beers': '749 Beers 1,805,250 Ratings',
  'brewery_ratings': '(3.919)'},
 {'brewery_name': 'Avery Brewing Co.',
  'brewery_location': 'Boulder, CO United States',
  'brewery_type': 'Macro Brewery',
  'number_beers': '532 Beers 2,240,081 Ratings',
  'brewery_ratings': '(3.891)'},
 {'brewery_name': 'Funky Buddha Brewery',
  'brewery_location': 'Oakland Park, FL United States',
  'brewery_type': 'Macro Brewery',
  'number_beers': '1,135 Beers 832,764 Ratings',
  'brewery_ratings': '(3.891)'},
 {'brewery_name': 'Goose Island Beer Co.',
  'brewery_location': 'Chicago, IL United States',
  'brewery_type': 'Macro Brewery',
  'number_beers': '1,37